# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [28]:
# Libraries

import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [29]:
admissions = pd.read_csv("data/Admission_Predict.csv")
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [30]:
admissions.columns = admissions.columns.str.rstrip()
admissions.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [31]:
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [32]:
admissions.isna().sum() # can also do admissions.isna().value_counts(); isna or isnull

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [33]:
admissions = admissions.set_index("Serial No.", drop=False)
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [34]:
len(admissions[["GRE Score","CGPA"]].value_counts()) == len(admissions)

True

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [35]:
admissions.set_index(["GRE Score","CGPA"], inplace=True)
admissions

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...
324,9.04,381,110,3,3.5,3.5,1,0.82
325,9.11,382,107,3,3.0,3.5,1,0.84
330,9.45,383,116,4,5.0,4.5,1,0.91


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [36]:
admissions.reset_index(inplace=True)
admissions

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91
383,312,8.78,384,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [37]:
admissions[(admissions["CGPA"] > 9) & (admissions["Research"] == 1)]

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
4,330,9.34,5,115,5,4.5,3.0,1,0.90
10,328,9.10,11,112,4,4.0,4.5,1,0.78
19,328,9.50,20,116,5,5.0,5.0,1,0.94
20,334,9.70,21,119,5,5.0,4.5,1,0.95
...,...,...,...,...,...,...,...,...,...
379,329,9.23,380,111,4,4.5,4.0,1,0.89
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [38]:
mean_chance_admit = admissions[(admissions["CGPA"] > 9) & (admissions["SOP"] < 3.5)]

mean_chance_admit["Chance of Admit"].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [39]:
# Libraries

import numpy as np

In [40]:
def standardize(column):
    """
    Take a column from a dataframe and subtrack the column's mean value
    then divide by the column's standard deviation in order to
    return a standardized column
    """
    
    return (column - np.mean(column))/np.std(column)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [41]:
admissions["CGPA_std"] = standardize(admissions["CGPA"])

admissions["GRE_std"] = standardize(admissions["GRE Score"])

admissions["LOR_std"] = standardize(admissions["LOR"])

admissions

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019


We will generate the decision choice at random using the code below. Please run the cell.

In [42]:
# Libraries

from random import choices

In [43]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [44]:
admissions["deciding_column"] = admissions.lookup(admissions.index, decision_choice)
admissions

# but lookup is deprecated

C:\Users\arigo\AppData\Local\Temp\ipykernel_9388\876983862.py:1: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  admissions["deciding_column"] = admissions.lookup(admissions.index, decision_choice)


,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.629546
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.852571
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.304036


In [45]:
admissions["deciding_column"] = [admissions._get_value(row, column) for row, column in zip(admissions.index, decision_choice)]
admissions

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.629546
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.852571
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.304036


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [46]:
admissions["decision"] = np.where(admissions["deciding_column"] > 0.8, 1, 0)
admissions

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.629546,0
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.852571,1
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729,1
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.304036,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [47]:
admissions["decision"].value_counts()

0    302
1     83
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [48]:
# libraries

import re

In [49]:
admissions.columns = [re.sub("\W", "", column.lower().replace(" ", "_")) for column in admissions.columns]
admissions

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.629546,0
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.852571,1
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729,1
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.304036,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [51]:
def boost(row):
    """
    Examines the University Rating and give +10 points on the GRE score
    if the University Rating is 4 or higher
    """
    if row["university_rating"] >= 4:
        return row["gre_score"] + 10
    else:
        return row["gre_score"]

In [52]:
admissions["adjusted_gre"] = admissions.apply(boost, axis=1)
admissions

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663,1,347.0
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0,316.0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297,0,322.0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,0,314.0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.629546,0,324.0
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.852571,1,325.0
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729,1,340.0
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.304036,0,312.0


In [53]:
admissions["adjusted_gre"] = pd.cut(admissions["adjusted_gre"], bins=4)

admissions["adjusted_gre"].value_counts()

(305.0, 320.0]     150
(320.0, 335.0]      90
(335.0, 350.0]      75
(289.94, 305.0]     70
Name: adjusted_gre, dtype: int64